In [ ]:
import numpy as np
import lightkurve as lk
from transitleastsquares import transitleastsquares
import warnings
import seaborn as sns
from scipy.ndimage import gaussian_filter1d
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import gridplot
from bokeh.models import Span, Label, HoverTool
import matplotlib.pyplot as plt
from astroquery.mast import Catalogs

# Import the EXOTIC light curve fitter (ensure EXOTIC is installed)
from exotic.api.elca import lc_fitter

%matplotlib ipympl
output_notebook()
warnings.simplefilter('ignore')

# Global data arrays
all_time = []
all_flux = []

# --- User-set default dilution factor (used if catalog query fails)
default_dilution_factor = 0.11

# --- New Global Settings ---
DOWNSAMPLE_THRESHOLD = 5000  # if number of points > threshold then downsample
BIN_FACTOR = 5                # factor by which to average data points
COARSE_PERIOD_STEPS = 50      # number of steps for coarse TLS search
REFINE_PERIOD_STEPS = 200     # number of steps for the refined search
VERBOSE = True                # flag for printing progress messages

# --- Stellar Parameters ---
STAR_MASS_SUN = 0.405      # in solar masses (update as needed)
STAR_RADIUS_SUN = 0.406    # in solar radii (update as needed)

# --- Utility Functions ---
def gaussian_filter(data, sigma=3.0):
    return gaussian_filter1d(data, sigma=sigma)

def clean_data(time, flux):
    mask = np.isfinite(time) & np.isfinite(flux)
    time_clean = time[mask]
    flux_clean = flux[mask]
    flux_normalized = flux_clean / np.nanmedian(flux_clean)
    return time_clean, flux_normalized

def downsample_data(time, flux, factor):
    n_points = len(time)
    n_bins = n_points // factor
    binned_time = np.mean(time[:n_bins*factor].reshape(-1, factor), axis=1)
    binned_flux = np.mean(flux[:n_bins*factor].reshape(-1, factor), axis=1)
    return binned_time, binned_flux

def process_sector(sector, tic_id):
    try:
        print(f"Processing light curve for sector {sector}...")
        search_result = lk.search_lightcurve(f'TIC {tic_id}', mission='TESS', sector=sector)
        if len(search_result) > 0:
            lc_file = search_result.download()
            flux = lc_file.flux.value
            time = lc_file.time.value
            mask = ~np.isnan(flux)
            time = time[mask]
            flux = flux[mask] / np.nanmedian(flux[mask]) if np.nanmedian(flux[mask]) != 0 else flux[mask]
            return time, flux
        else:
            print(f"No light curve data found for sector {sector}")
            return None, None
    except Exception as e:
        print(f"Error processing light curve for sector {sector}: {e}")
        return None, None

def get_dilution_factor(tic_id, search_radius=21):
    try:
        target_table = Catalogs.query_criteria(catalog="TIC", ID=tic_id)
        if len(target_table) == 0:
            print("Target not found in TIC; using default dilution factor.")
            return default_dilution_factor
        target = target_table[0]
        ra, dec = target["ra"], target["dec"]
        radius_deg = search_radius / 3600.0
        neighbors = Catalogs.query_region(f"{ra} {dec}", radius=radius_deg, catalog="TIC")
        neighbors = neighbors[neighbors["ID"] != tic_id]
        if len(neighbors) == 0:
            return 0.0
        target_flux = 10**(-0.4 * target["Tmag"])
        neighbor_flux = np.sum(10**(-0.4 * neighbors["Tmag"]))
        dilution_factor = neighbor_flux / (target_flux + neighbor_flux)
        print(f"Calculated dilution factor for TIC {tic_id} = {dilution_factor:.4f}")
        return dilution_factor
    except Exception as e:
        print("Error computing dilution factor:", e)
        return default_dilution_factor

def calculate_t0_uncertainty(time, tls_results, period):
    return np.std(tls_results.power) / period * 0.1

def calculate_inclination_and_impact(tls_results, semi_major_axis, star_radius):
    return 90.0, 0.0, 0.5, 0.05

def calculate_semi_major_axis(period, star_mass, period_uncertainty, G=6.67430e-11, solar_mass=1.989e30):
    star_mass_kg = star_mass * solar_mass
    period_seconds = period * 86400.
    semi_major_axis_m = ((G * star_mass_kg * period_seconds**2) / (4 * np.pi**2))**(1/3)
    a_AU = semi_major_axis_m / 1.496e11
    a_uncertainty = (2/3) * (period_uncertainty / period) * a_AU
    return a_AU, a_uncertainty

def classify_planet_type(radius_km):
    radius_solar = radius_km / 695700.0
    if radius_solar >= 0.12:
        return 'Red Dwarf Star'
    if radius_km < 0.5 * 6371:
        return 'Mercury-like'
    elif radius_km < 0.9 * 6371:
        return 'Venus-like'
    elif radius_km < 1.5 * 6371:
        return 'Earth-like'
    elif radius_km < 2 * 6371:
        return 'Super-Earth'
    elif radius_km < 3 * 6371:
        return 'Mini-Neptune'
    elif radius_km < 4 * 6371:
        return 'Neptune-like'
    else:
        return 'Gas Giant (Jupiter/Saturn-like)'

def estimate_planet_mass(planet_radius_km, planet_radius_uncertainty_km, host_star_radius_km=None):
    R_earth = 6378.0
    solar_radius_km = 695700.0
    candidate_radius_solar = planet_radius_km / solar_radius_km
    if host_star_radius_km is not None:
        if planet_radius_km > 0.5 * host_star_radius_km or candidate_radius_solar >= 0.12:
            mass_solar = candidate_radius_solar
            mass_earth = mass_solar * 332946.0
            mass_jup = mass_solar * 1047.0
            frac_unc = planet_radius_uncertainty_km / planet_radius_km
            return mass_earth, mass_earth * frac_unc, mass_jup, mass_jup * frac_unc
    planet_radius_earth = planet_radius_km / R_earth
    sigma_R_earth = planet_radius_uncertainty_km / R_earth
    if planet_radius_earth < 1.5:
        mass_earth = planet_radius_earth**3.7
        mass_earth_uncertainty = 3.7 * planet_radius_earth**2.7 * sigma_R_earth
    elif planet_radius_earth < 2.5:
        mass_earth = 2.7 * (planet_radius_earth**1.3)
        mass_earth_uncertainty = 2.7 * 1.3 * planet_radius_earth**0.3 * sigma_R_earth
    elif planet_radius_earth < 4.5:
        mass_earth = 1.13 * (planet_radius_earth**2.0)
        mass_earth_uncertainty = 1.13 * 2.0 * planet_radius_earth**1.0 * sigma_R_earth
    elif planet_radius_earth < 6.0:
        mass_earth = 1.5 * (planet_radius_earth**2.0)
        mass_earth_uncertainty = 1.5 * 2.0 * planet_radius_earth**1.0 * sigma_R_earth
    elif planet_radius_earth < 10.0:
        mass_earth = planet_radius_earth**2.06
        mass_earth_uncertainty = 2.06 * planet_radius_earth**1.06 * sigma_R_earth
    elif planet_radius_earth < 20.0:
        mass_earth = planet_radius_earth**2.38
        mass_earth_uncertainty = 2.38 * planet_radius_earth**1.38 * sigma_R_earth
    else:
        mass_jup = 10.0
        mass_jup_uncertainty = 2.0
        mass_earth = mass_jup * 317.8
        mass_earth_uncertainty = mass_jup_uncertainty * 317.8
        return mass_earth, mass_earth_uncertainty, mass_jup, mass_jup_uncertainty
    mass_jup = mass_earth / 317.8
    mass_jup_uncertainty = mass_earth_uncertainty / 317.8
    return mass_earth, mass_earth_uncertainty, mass_jup, mass_jup_uncertainty

def advanced_true_anomaly(t, t0, period, eccentricity, omega):
    t = np.atleast_1d(t)
    M = 2*np.pi*(t - t0)/period
    E = M.copy()
    for _ in range(50):
        delta = (E - eccentricity*np.sin(E) - M) / (1 - eccentricity*np.cos(E))
        E = E - delta
        if np.all(np.abs(delta) < 1e-10):
            break
    f = 2*np.arctan2(np.sqrt(1+eccentricity)*np.sin(E/2), np.sqrt(1-eccentricity)*np.cos(E/2))
    f_deg = np.degrees(f + np.radians(omega))
    if f_deg.size == 1:
        return f_deg[0]
    return f_deg

def calculate_planet_radius(radius_ratio, star_radius_km, is_binary=False, star_radius2_km=None, flux_ratio=None):
    if flux_ratio is not None and flux_ratio > 0:
        corrected_radius_ratio = radius_ratio / np.sqrt(1-flux_ratio)
    else:
        corrected_radius_ratio = radius_ratio
    if is_binary and star_radius2_km is not None:
        combined_star_radius_km = np.sqrt(star_radius_km**2 + star_radius2_km**2)
    else:
        combined_star_radius_km = star_radius_km
    planet_radius_km = corrected_radius_ratio * combined_star_radius_km
    planet_radius_jup = planet_radius_km / 71492
    planet_type = classify_planet_type(planet_radius_km)
    return planet_radius_km, planet_radius_jup, planet_type

def calculate_transit_parameters(tls_results, flux, star_radius, star_mass, period_uncertainty, is_binary=True, star_radius2=None, dilution_factor=0.0):
    flux_median = np.nanmedian(flux)
    flux_min = np.min(flux)
    measured_depth = (flux_median - flux_min)/flux_median
    if dilution_factor > 0:
        transit_depth = measured_depth/(1-dilution_factor)
    else:
        transit_depth = measured_depth
    transit_depth_uncertainty = np.std(flux-flux_median)/flux_median
    if transit_depth <= 0:
        raise ValueError("Transit depth is non-positive.")
    radius_ratio = np.sqrt(transit_depth)
    radius_ratio_uncertainty = (transit_depth_uncertainty/(2*np.sqrt(transit_depth))) if transit_depth > 0 else np.nan
    period = tls_results.period
    t0 = tls_results.T0
    duration = tls_results.duration
    depth_ppt = transit_depth*1e3
    depth_ppm = transit_depth*1e6
    depth_uncertainty_ppt = transit_depth_uncertainty*1e3
    depth_uncertainty_ppm = transit_depth_uncertainty*1e6
    semi_major_axis, semi_major_axis_uncertainty = calculate_semi_major_axis(period, star_mass, period_uncertainty)
    inclination, impact_parameter, inclination_unc, impact_parameter_unc = calculate_inclination_and_impact(tls_results, semi_major_axis, star_radius)
    planet_radius_km, planet_radius_jup, planet_type = calculate_planet_radius(radius_ratio, star_radius*696340, is_binary=is_binary, star_radius2_km=star_radius2, flux_ratio=dilution_factor)
    planet_radius_uncertainty_km = radius_ratio_uncertainty*(star_radius*696340)
    mass_earth, mass_earth_unc, mass_jup, mass_jup_unc = estimate_planet_mass(planet_radius_km, planet_radius_uncertainty_km, host_star_radius_km=star_radius*696340)
    return {
        "transit_depth": transit_depth,
        "transit_depth_uncertainty": transit_depth_uncertainty,
        "depth_ppt": depth_ppt,
        "depth_ppm": depth_ppm,
        "depth_uncertainty_ppt": depth_uncertainty_ppt,
        "depth_uncertainty_ppm": depth_uncertainty_ppm,
        "radius_ratio": radius_ratio,
        "radius_ratio_uncertainty": radius_ratio_uncertainty,
        "period": period,
        "t0": t0,
        "duration": duration,
        "semi_major_axis": semi_major_axis,
        "semi_major_axis_uncertainty": semi_major_axis_uncertainty,
        "inclination": inclination,
        "inclination_uncertainty": inclination_unc,
        "impact_parameter": impact_parameter,
        "impact_parameter_uncertainty": impact_parameter_unc,
        "planet_radius_km": planet_radius_km,
        "planet_radius_jup": planet_radius_jup,
        "planet_type": planet_type,
        "planet_mass_earth": mass_earth,
        "planet_mass_earth_uncertainty": mass_earth_unc,
        "planet_mass_jup": mass_jup,
        "planet_mass_jup_uncertainty": mass_jup_unc
    }

def plot_in_out_transits(time, flux, period, t0, duration):
    phases = ((time - t0 + 0.5*period) % period)-0.5*period
    in_transit = np.abs(phases) < (duration/2)
    out_transit = ~in_transit
    fig_in_out = figure(title='In-Transit vs Out-of-Transit Data',
                        x_axis_label='Time (days)', y_axis_label='Relative Flux',
                        width=800, height=400)
    fig_in_out.circle(time[out_transit], flux[out_transit], size=3, color='blue', legend_label='Out-of-Transit')
    fig_in_out.circle(time[in_transit], flux[in_transit], size=3, color='red', legend_label='In-Transit')
    fig_in_out.add_tools(HoverTool(tooltips=[("Time", "@x"), ("Flux", "@y")]))
    return fig_in_out

def display_tpf(tic_id, sector):
    try:
        tpf_search = lk.search_targetpixelfile(f'TIC {tic_id}', mission='TESS', sector=sector)
        if len(tpf_search) > 0:
            tpf = tpf_search.download()
            tpf.plot(aperture_mask=True)
            plt.title(f'TPF for TIC {tic_id} Sector {sector}')
            plt.show()
        else:
            print(f"No TPF found for TIC {tic_id} in sector {sector}.")
    except Exception as e:
        print(f"Error displaying TPF for sector {sector}: {e}")

def plot_phase_folded_lightcurve(tls_results, time, flux, bins=500):
    period = tls_results.period
    t0 = tls_results.T0
    model_time = tls_results.model_lightcurve_time
    model_flux = tls_results.model_lightcurve_model
    phase = ((time - t0 + 0.5*period) % period)-0.5*period
    sort_idx = np.argsort(phase)
    phase_sorted = phase[sort_idx]
    flux_sorted = flux[sort_idx]
    model_phase = ((model_time - t0 + 0.5*period) % period)-0.5*period
    model_sort_idx = np.argsort(model_phase)
    model_phase_sorted = model_phase[model_sort_idx]
    model_flux_sorted = model_flux[model_sort_idx]
    phase_bins = np.linspace(-0.5*period, 0.5*period, bins+1)
    binned_phase, binned_flux = [], []
    for i in range(len(phase_bins)-1):
        in_bin = (phase_sorted>=phase_bins[i]) & (phase_sorted<phase_bins[i+1])
        if np.any(in_bin):
            binned_phase.append(np.mean(phase_sorted[in_bin]))
            binned_flux.append(np.mean(flux_sorted[in_bin]))
    binned_phase = np.array(binned_phase)
    binned_flux = np.array(binned_flux)
    model_flux_interp = np.interp(phase_sorted, model_phase_sorted, model_flux_sorted)
    residuals = flux_sorted - model_flux_interp
    fig, (ax_top, ax_bottom) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios':[3,1]}, figsize=(8,6))
    ax_top.scatter(phase_sorted, flux_sorted, s=3, color='gray', alpha=0.4, label='Data')
    ax_top.scatter(binned_phase, binned_flux, s=3, color='red', edgecolor='black', zorder=5, label='Binned')
    ax_top.plot(model_phase_sorted, model_flux_sorted, color='orange', lw=2, label='TLS Model')
    ax_top.set_ylabel("Relative Flux")
    ax_top.set_title(f"Phase-folded Light Curve (P = {period:.5f} d)")
    ax_top.legend(loc='best')
    ax_top.set_xlim(-0.10, 0.10)
    ax_bottom.scatter(phase_sorted, residuals, s=5, color='gray', alpha=0.6)
    ax_bottom.axhline(0, color='red', lw=1)
    ax_bottom.set_xlabel("Phase (days)")
    ax_bottom.set_ylabel("Residuals")
    ax_bottom.set_xlim(-0.3, 0.3)
    plt.tight_layout()
    plt.show()

def plot_tls_periodogram(main_tls, main_params, tic_id):
    fig = figure(title=f'TLS Periodogram for TIC {tic_id}', x_axis_label='Period (days)', y_axis_label='SNR', width=800, height=400)
    fig.line(main_tls.periods, main_tls.power, line_width=1, line_color='green', legend_label='TLS Power')
    best_period = main_params["period"]
    best_SNR = max(main_tls.power)
    span = Span(location=best_period, dimension='height', line_color='green', line_dash='dashed', line_width=1.5)
    fig.add_layout(span)
    label = Label(x=best_period, y=best_SNR, text=f"Main: {best_period:.6f} d (SNR={best_SNR:.1f})", text_color="green", text_font_size="8pt")
    fig.add_layout(label)
    fig.add_tools(HoverTool(tooltips=[("Period", "@x"), ("SNR", "@y")]))
    show(fig, notebook_handle=True)
    push_notebook()

def plot_detrended_lightcurve_with_transits(time, flux, period, t0, duration):
    plt.figure(figsize=(10,4))
    plt.plot(time, flux, 'k.', markersize=2, label='Detrended Data')
    transit_times = []
    t_min, t_max = np.min(time), np.max(time)
    n_before = int(np.floor((t_min-t0)/period))
    n_after = int(np.ceil((t_max-t0)/period))
    for n in range(n_before, n_after+1):
        transit_time = t0 + n*period
        if t_min <= transit_time <= t_max:
            transit_times.append(transit_time)
            plt.axvline(x=transit_time, color='green', linestyle='dashed', lw=1)
    plt.xlabel("Time (days)")
    plt.ylabel("Normalized Flux")
    plt.title("Detrended Light Curve with Transit Markers")
    plt.legend()
    plt.tight_layout()
    plt.show()

def run_tls_search(time, flux, period_min, period_max):
    original_length = len(time)
    if original_length > DOWNSAMPLE_THRESHOLD:
        print(f"Data length ({original_length}) exceeds threshold. Downsampling by factor {BIN_FACTOR}.")
        time, flux = downsample_data(time, flux, BIN_FACTOR)
    else:
        print("No downsampling needed.")
    print("Running coarse TLS search...")
    model = transitleastsquares(time, flux)
    coarse_result = model.power(period_min=period_min, period_max=period_max, period_steps=COARSE_PERIOD_STEPS, show_progress_bar=VERBOSE)
    if coarse_result.periods.size == 0:
        raise ValueError("No significant periods found in coarse search.")
    best_period_coarse = coarse_result.period
    refine_window = 0.1 * best_period_coarse
    new_period_min = max(period_min, best_period_coarse - refine_window)
    new_period_max = min(period_max, best_period_coarse + refine_window)
    print(f"Refining TLS search around best period {best_period_coarse:.5f} d between {new_period_min:.5f} and {new_period_max:.5f} d")
    refined_result = model.power(period_min=new_period_min, period_max=new_period_max, period_steps=REFINE_PERIOD_STEPS, show_progress_bar=VERBOSE)
    return refined_result

def compute_a_over_rstar_from_kepler(period_days, star_mass_sun, star_radius_sun):
    MSUN_KG = 1.98847e30
    RSUN_M = 6.95700e8
    G = 6.67430e-11
    period_s = period_days * 86400.0
    a_m = (G * (star_mass_sun * MSUN_KG) * period_s**2 / (4 * np.pi**2))**(1/3)
    a_over_r = a_m / (star_radius_sun * RSUN_M)
    return a_over_r

def main():
    tic_id = "4918918"  # example TIC ID; update as needed
    sectors = [21, 48]        # example sector; update as needed
    global all_time, all_flux
    dilution_factor = get_dilution_factor(tic_id, search_radius=21)
    for sector in sectors:
        time, flux = process_sector(sector, tic_id)
        if time is not None and flux is not None:
            all_time.append(time)
            all_flux.append(flux)
        display_tpf(tic_id, sector)
    if not all_time or not all_flux:
        print("No valid light curve data available after processing.")
        return
    all_time_np = np.concatenate(all_time)
    all_flux_np = np.concatenate(all_flux)
    all_time_np, all_flux_np = clean_data(all_time_np, all_flux_np)
    lc = lk.LightCurve(time=all_time_np, flux=all_flux_np)
    flat_lc = lc.flatten(window_length=401)
    all_time_np = flat_lc.time.value
    all_flux_np = flat_lc.flux.value
    ld_coeffs = [0, 0]
    try:
        PERIOD_MIN = 19  # days
        PERIOD_MAX = 20   # days
        tls_results = run_tls_search(all_time_np, all_flux_np, PERIOD_MIN, PERIOD_MAX)
        if tls_results.periods.size > 0:
            period = tls_results.period
            period_uncertainty = tls_results.period_uncertainty
            t0 = tls_results.T0
            duration = tls_results.duration
            transit_params = calculate_transit_parameters(
                tls_results, all_flux_np, star_radius=0.406, star_mass=0.405,
                period_uncertainty=period_uncertainty,
                dilution_factor=dilution_factor, is_binary=True, star_radius2=0.0
            )
            # Force transit midtime into full BJD (i.e., add 2457000)
            tmid_full = t0 + 2457000.0
            print("Orbital Parameters:")
            print(f"  Period: {transit_params['period']:.6f} ± {period_uncertainty:.6f} days")
            print(f"  Transit Midpoint (t0): {tmid_full:.6f} days ± {calculate_t0_uncertainty(all_time_np, tls_results, period):.6f}")
            print(f"  Duration: {transit_params['duration']*24:.2f} hours")
            print(f"  Semi-major Axis (a): {transit_params['semi_major_axis']:.4f} ± {transit_params['semi_major_axis_uncertainty']:.4f} AU")
            print(f"  Transit Depth: {transit_params['depth_ppt']:.2f} ± {transit_params['depth_uncertainty_ppt']:.2f} ppt, {transit_params['depth_ppm']:.2f} ± {transit_params['depth_uncertainty_ppm']:.2f} ppm")
            print(f"  Radius Ratio (Rp/Rs): {transit_params['radius_ratio']:.4f} ± {transit_params['radius_ratio_uncertainty']:.4f}")
            print(f"  Candidate Radius: {transit_params['planet_radius_km']:.2f} km, {transit_params['planet_radius_jup']:.4f} R_Jup")
            print(f"  Candidate Type: {transit_params['planet_type']}")
            print(f"  Synthetic Companion Mass: {transit_params['planet_mass_earth']:.2f} ± {transit_params['planet_mass_earth_uncertainty']:.2f} M_Earth, {transit_params['planet_mass_jup']:.4f} ± {transit_params['planet_mass_jup_uncertainty']:.4f} M_Jup")
            plot_phase_folded_lightcurve(tls_results, all_time_np, all_flux_np, bins=100)
            plot_tls_periodogram(tls_results, transit_params, tic_id)
            plot_detrended_lightcurve_with_transits(all_time_np, all_flux_np, period, t0, duration)
            
            print("\nStarting fully automated EXOTIC phase-folded transit fitting...")
            # Always use a fixed offset of 2457000.0 so that tmid is in full BJD
            t_offset = 2457000.0
            
            # Compute a/R* from Kepler's Third Law:
            auto_ars = compute_a_over_rstar_from_kepler(period, STAR_MASS_SUN, STAR_RADIUS_SUN)
            print(f"Auto-computed a/R* = {auto_ars:.2f}")
            
            # Instead of relying on TLS t0, let EXOTIC fit for mid-transit.
            # Set an initial tmid using TLS but with wide bounds, converted to full BJD.
            tmid_init = t0 + t_offset
            
            # Phase-fold the light curve (for fitting) using TLS period and t0.
            folded_phase = ((all_time_np - t0 + 0.5*period) % period) / period - 0.5
            folded_time = folded_phase * period + t0
            pdur = 2 * np.arctan(1.0/auto_ars) / (2*np.pi)
            transit_mask = np.abs(folded_phase) < 2*pdur
            if np.sum(transit_mask) == 0:
                raise Exception("No phase-folded data within the transit window found.")
            fit_time = folded_time[transit_mask]
            fit_flux = all_flux_np[transit_mask]
            flux_err = np.full_like(fit_flux, np.std(fit_flux))
            airmass = np.zeros_like(fit_time)
            fit_time = np.array(fit_time, dtype=float)
            fit_flux = np.array(fit_flux, dtype=float)
            flux_err = np.array(flux_err, dtype=float)
            airmass = np.array(airmass, dtype=float)
            
            # Build transit parameter dictionary for EXOTIC.
            rprs_est = np.sqrt(transit_params["transit_depth"])
            tpars = {
                "rprs": rprs_est,
                "ars": auto_ars,
                "per": period,
                "inc": 90.0,         # initial guess; allow EXOTIC to adjust inclination
                "tmid": tmid_init,   # initial guess (full BJD)
                "omega": 0.0,
                "ecc": 0.0,
                "a1": 0.0,
                "a2": 0.0,
                "u0": 0.0,
                "u1": 0.0,
                "u2": 0.0,
                "u3": 0.0
            }
            # Set wide bounds for tmid so that EXOTIC can fit it independently.
            mybounds = {
                "rprs": [0, 3*rprs_est],
                "tmid": [tmid_init - 0.5, tmid_init + 0.5],
                "inc": [80, 90],
                "ars": [auto_ars*0.8, auto_ars*1.2]
            }
            try:
                phase_fit = lc_fitter(fit_time + t_offset, fit_flux, flux_err, airmass, tpars, mybounds)
            except Exception as e:
                print("EXOTIC phase-folded fitting failed:", e)
                phase_fit = None
            if phase_fit is not None:
                print("\nEXOTIC Phase-Folded Fit Results:")
                for key in phase_fit.bounds.keys():
                    print(f"{key}: {phase_fit.parameters[key]:.6f} ± {phase_fit.errors[key]}")
                fig_best = phase_fit.plot_bestfit(title="EXOTIC Phase-Folded Fit", bin_dt=0.5/24.)
                plt.show()
                fig_triangle = phase_fit.plot_triangle()
                plt.show()
            else:
                print("EXOTIC phase-folded fitting did not produce valid results.")
        else:
            print("No significant periods found in TLS analysis.")
    except Exception as e:
        print(f"Error during TLS analysis: {e}")

if __name__ == "__main__":
    main()

C:\Users\kylel\anaconda3\Lib\site-packages\lightkurve\config\__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to C:\Users\kylel\.lightkurve\cache. Please move all the files in the legacy directory C:\Users\kylel\.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...


Loading BokehJS ...

Calculated dilution factor for TIC 4918918 = 0.0047
Processing light curve for sector 21...
